# Precomputed Perlin Noise Vector Field

A circular path in noise space drives a time-dependent 2D field. The notebook now enforces a top-down flow: setup, simulation, cache loading, and rendering. The simulation runs independently from rendering so cached data can be reused across renders.

In [ ]:
%load_ext manim.utils.ipython_magic

## Imports and Perlin sampler

Helper utilities are defined up front so later cells only orchestrate simulation or rendering. The sampler is kept lightweight to avoid extra dependencies.

In [ ]:
from __future__ import annotations

import math
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple

import numpy as np
from tqdm.auto import tqdm
from manim import (
    Scene,
    ValueTracker,
    Arrow,
    VGroup,
    RIGHT,
    color_gradient,
    ManimColor,
    FadeIn,
    FadeOut,
    smooth,
    BLUE_E,
    BLUE_C,
    GREEN_B,
    YELLOW_C,
    RED_C,
)


# Minimal Perlin sampler so the notebook has no extra runtime dependencies.
def _fade(t: np.ndarray | float) -> np.ndarray | float:
    return 6 * t**5 - 15 * t**4 + 10 * t**3


def _lerp(a: np.ndarray | float, b: np.ndarray | float, t: np.ndarray | float) -> np.ndarray | float:
    return a + t * (b - a)


@dataclass
class PerlinSampler:
    seed: int = 1

    def __post_init__(self) -> None:
        rng = np.random.default_rng(self.seed)
        perm = np.arange(256, dtype=int)
        rng.shuffle(perm)
        self.permutation = np.concatenate([perm, perm])

    def _gradient(self, hash_value: np.ndarray) -> np.ndarray:
        gradients = np.array(
            [
                [1, 1, 0],
                [-1, 1, 0],
                [1, -1, 0],
                [-1, -1, 0],
                [1, 0, 1],
                [-1, 0, 1],
                [1, 0, -1],
                [-1, 0, -1],
                [0, 1, 1],
                [0, -1, 1],
                [0, 1, -1],
                [0, -1, -1],
            ]
        )
        return gradients[hash_value % gradients.shape[0]]

    def __call__(self, x: float, y: float, z: float) -> float:
        x_arr = np.asarray(x, dtype=float)
        y_arr = np.asarray(y, dtype=float)
        z_arr = np.asarray(z, dtype=float)

        xi = np.floor(x_arr).astype(int) & 255
        yi = np.floor(y_arr).astype(int) & 255
        zi = np.floor(z_arr).astype(int) & 255
        xf, yf, zf = x_arr - np.floor(x_arr), y_arr - np.floor(y_arr), z_arr - np.floor(z_arr)
        u, v, w = _fade(xf), _fade(yf), _fade(zf)

        def gradient_at(dx: float, dy: float, dz: float, px: int, py: int, pz: int) -> float:
            hash_value = self.permutation[self.permutation[self.permutation[xi + px] + yi + py] + zi + pz]
            gradient = self._gradient(hash_value)
            return float(gradient[..., 0] * dx + gradient[..., 1] * dy + gradient[..., 2] * dz)

        n000 = gradient_at(float(xf), float(yf), float(zf), 0, 0, 0)
        n100 = gradient_at(float(xf - 1), float(yf), float(zf), 1, 0, 0)
        n010 = gradient_at(float(xf), float(yf - 1), float(zf), 0, 1, 0)
        n110 = gradient_at(float(xf - 1), float(yf - 1), float(zf), 1, 1, 0)
        n001 = gradient_at(float(xf), float(yf), float(zf - 1), 0, 0, 1)
        n101 = gradient_at(float(xf - 1), float(yf), float(zf - 1), 1, 0, 1)
        n011 = gradient_at(float(xf), float(yf - 1), float(zf - 1), 0, 1, 1)
        n111 = gradient_at(float(xf - 1), float(yf - 1), float(zf - 1), 1, 1, 1)

        x1 = _lerp(n000, n100, u)
        x2 = _lerp(n010, n110, u)
        x3 = _lerp(n001, n101, u)
        x4 = _lerp(n011, n111, u)
        y1 = _lerp(x1, x2, v)
        y2 = _lerp(x3, x4, v)
        return float(_lerp(y1, y2, w))

## Simulation configuration

These values are centralized for quick iteration. Adjusting the config only affects the simulation stage, keeping rendering deterministic from cached data.

In [ ]:
@dataclass
class SimulationConfig:
    output_path: Path = Path("notebooks/data/perlin_vector_field_cache.npz")
    duration: float = 8.0
    frame_rate: int = 30
    grid_x_span: Tuple[float, float] = (-4.5, 4.5)
    grid_y_span: Tuple[float, float] = (-2.5, 2.5)
    grid_x_count: int = 20
    grid_y_count: int = 14
    noise_seed: int = 7
    noise_scale: float = 0.6
    noise_radius: float = 0.8
    noise_loop_count: int = 2
    noise_wobble_scale: float = 0.35
    noise_wobble_harmonic: int = 2
    noise_vertical_harmonic: int = 3
    arrow_length: float = 0.6
    color_bias: float = 0.2
    color_gain: float = 0.8
    palette: tuple[ManimColor, ...] = tuple(color_gradient([BLUE_E, BLUE_C, GREEN_B, YELLOW_C, RED_C], 64))


config = SimulationConfig()


## Map grid points into noise space

Each grid anchor is scaled into noise coordinates, offset by a circular orbit with angle $\theta(t)$ that completes an integer number of turns over the clip, and warped by periodic sinusoids so every time-dependent input repeats after the configured duration.


In [ ]:
noise_sampler = PerlinSampler(seed=config.noise_seed)


def angular_position(time: float, *, cfg: SimulationConfig = config) -> float:
    """Map wall-clock time onto the angular coordinate of the noise-space orbit."""
    return math.tau * cfg.noise_loop_count * (time / cfg.duration)


def sample_vector_field(point: np.ndarray, time: float, *, cfg: SimulationConfig = config) -> tuple[np.ndarray, float]:
    px, py = point[:2]
    theta = angular_position(time, cfg=cfg)
    noise_center = np.array([math.cos(theta), math.sin(theta), 0.0]) * cfg.noise_radius
    temporal_warp = np.array(
        [
            math.cos(theta * cfg.noise_wobble_harmonic),
            math.sin(theta * cfg.noise_wobble_harmonic),
            math.sin(theta * cfg.noise_vertical_harmonic),
        ]
    ) * cfg.noise_wobble_scale
    domain = np.array([px, py, 0.0]) * cfg.noise_scale + noise_center + temporal_warp

    dx = noise_sampler(*(domain + np.array([0.0, 0.0, 0.5])))
    dy = noise_sampler(*(domain + np.array([2.3, -1.7, -0.25])))
    vector = np.array([dx, dy])
    magnitude = float(np.linalg.norm(vector))
    if magnitude > 1e-6:
        vector = vector / magnitude
    vector *= cfg.arrow_length
    return vector, magnitude


def color_from_magnitude(magnitude: float, *, cfg: SimulationConfig = config) -> ManimColor:
    scaled = float(np.clip(magnitude * cfg.color_gain + cfg.color_bias, 0, 1))
    index = int(scaled * (len(cfg.palette) - 1))
    return cfg.palette[index]


## Stage 1: simulate and cache the field

Sample the Perlin field on a fixed time grid over $[0, 	ext{duration})$, compute vectors for each anchor, and save the resulting positions and colors to disk. Skipping the terminal time avoids duplicating the first frame when the video loops.


In [ ]:
def run_simulation(cfg: SimulationConfig = config) -> Path:
    frame_dt = 1.0 / cfg.frame_rate
    times = np.linspace(0.0, cfg.duration, int(cfg.duration * cfg.frame_rate), endpoint=False)
    xs = np.linspace(*cfg.grid_x_span, cfg.grid_x_count)
    ys = np.linspace(*cfg.grid_y_span, cfg.grid_y_count)
    anchors = np.array([[x, y, 0.0] for x in xs for y in ys], dtype=np.float32)

    starts = np.repeat(anchors[None, ...], len(times), axis=0)
    ends = np.zeros_like(starts)
    magnitudes = np.zeros((len(times), len(anchors)), dtype=np.float32)
    colors = np.empty((len(times), len(anchors)), dtype="<U7")

    for frame_index, t in enumerate(tqdm(times, desc="Simulating frames")):
        for anchor_index, anchor in enumerate(anchors):
            vector, magnitude = sample_vector_field(anchor, float(t), cfg=cfg)
            ends[frame_index, anchor_index] = anchor + np.append(vector, 0.0)
            magnitudes[frame_index, anchor_index] = magnitude
            colors[frame_index, anchor_index] = color_from_magnitude(magnitude, cfg=cfg).to_hex()

    cfg.output_path.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        cfg.output_path,
        times=times.astype(np.float32),
        anchors=anchors,
        starts=starts,
        ends=ends,
        magnitudes=magnitudes,
        colors=colors,
        frame_dt=np.float32(frame_dt if len(times) > 1 else 0.0),
    )
    return cfg.output_path


# Uncomment to regenerate the cache when parameters change.
# simulation_path = run_simulation()


## Stage 2: load cached data

If the cache is missing, regenerate it automatically so the render path always has data to consume.

In [ ]:
def load_cached_simulation(cfg: SimulationConfig = config) -> dict[str, np.ndarray]:
    if not cfg.output_path.exists():
        run_simulation(cfg)
    cache = np.load(cfg.output_path)
    return {
        "anchors": cache["anchors"],
        "starts": cache["starts"],
        "ends": cache["ends"],
        "colors": cache["colors"],
        "frame_dt": float(cache["frame_dt"]),
    }

## Stage 3: render from cached data

The Manim scene only indexes into cached arrays while animating, avoiding repeated sampling during rendering ([Manim Scene reference](https://docs.manim.community/en/stable/reference/manim.scene.scene.Scene.html)).

In [ ]:
class PerlinNoiseVectorField(Scene):
    def construct(self) -> None:
        cache = load_cached_simulation()
        anchors = cache["anchors"]
        starts = cache["starts"]
        ends = cache["ends"]
        colors = cache["colors"]
        frame_dt = cache["frame_dt"]
        total_frames = starts.shape[0]

        def make_arrow(anchor: np.ndarray) -> Arrow:
            return Arrow(
                anchor,
                anchor + RIGHT * 0.01,
                buff=0,
                max_stroke_width_to_length_ratio=2.5,
                max_tip_length_to_length_ratio=0.35,
                stroke_width=3.0,
            )

        frame_tracker = ValueTracker(0)

        def update_arrow(arrow: Arrow, anchor_index: int) -> None:
            frame_index = min(int(round(frame_tracker.get_value())), total_frames - 1)
            arrow.put_start_and_end_on(starts[frame_index, anchor_index], ends[frame_index, anchor_index])
            arrow.set_color(ManimColor(colors[frame_index, anchor_index]))

        arrow_list: list[Arrow] = [make_arrow(anchor) for anchor in anchors]
        arrows = VGroup(*arrow_list)

        for index, arrow in enumerate(arrow_list):
            arrow.add_updater(lambda mob, dt, idx=index: update_arrow(mob, idx))

        self.play(FadeIn(arrows), run_time=1.5, rate_func=smooth)

        def advance_frame(mob: ValueTracker, dt: float) -> None:
            mob.increment_value(dt / frame_dt)

        frame_tracker.add_updater(advance_frame)
        self.wait(frame_dt * total_frames)

        for arrow in arrow_list:
            arrow.clear_updaters()
        frame_tracker.clear_updaters()

        self.play(FadeOut(arrows), run_time=1.0, rate_func=smooth)


## Render the scene

Use the Manim IPython magic to generate and display the animation inline. Adjust quality flags as needed.

In [ ]:
%manim -qm -v WARNING PerlinNoiseVectorField